In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import multiprocessing
import os
import time
import psutil

def square_number(number):
    result = number ** 2
    cpu_id = psutil.Process().cpu_num()
    print(f"Child process (PID, time, CPU): \
        {os.getpid(), int(time.time()), cpu_id}  \
        The square of {number} is {result}")
    time.sleep(1)


def main():
    print(f"Main process (PID: {os.getpid()} \
        Starting parallel processes...")
    numbers_to_square = [5, 7, 10, 12]
    processes = []

    for num in numbers_to_square:
        process = multiprocessing.Process(target=square_number,  
                args=(num,))
        processes.append(process)
        process.start()

    # Now, wait for all processes to complete
    for process in processes:
        process.join()

    print(f"Main process (PID: {os.getpid()}):  \
        All parallel processes finished.")

if __name__ == '__main__':
    main()

Main process (PID: 17         Starting parallel processes...
Child process (PID, time, CPU):         (43, 1768471270, 1)          The square of 5 is 25
Child process (PID, time, CPU):         (46, 1768471270, 0)          The square of 7 is 49
Child process (PID, time, CPU):         (51, 1768471270, 1)          The square of 10 is 100Child process (PID, time, CPU):         (56, 1768471270, 0)          The square of 12 is 144

Main process (PID: 17):          All parallel processes finished.


In [3]:
import multiprocessing as mp
import random

def monte_carlo_task(samples):
    """Heavy computation: Estimate points inside a circle."""
    hits = 0
    for _ in range(samples):
        x, y = random.random(), random.random()
        if x**2 + y**2 <= 1.0:
            hits += 1
    return hits

if __name__ == "__main__":
    total_samples = 100_000
    num_tasks = 20
    samples_per_task = total_samples // num_tasks

    print("Master process: Starting a synchronous pool...")

    with mp.Pool(processes=mp.cpu_count()) as pool:
        # Use pool.map for synchronous execution
        # It will block until all tasks are complete and return a list of results
        results = pool.map(monte_carlo_task, (samples_per_task,) * num_tasks)

    print("Master process: All tasks completed synchronously.")

    # Final reduction (HPC logic)
    pi_est = (4.0 * sum(results)) / total_samples
    print(f"Final Pi Estimate: {pi_est}")

Master process: Starting a synchronous pool...
Master process: All tasks completed synchronously.
Final Pi Estimate: 3.14376


In [4]:
import multiprocessing as mp
import random

def monte_carlo_task(samples):
    hits = 0
    for _ in range(samples):
        x, y = random.random(), random.random()
        if x**2 + y**2 <= 1.0:
            hits += 1
    return hits

if __name__ == "__main__":
    total_samples = 100_000
    num_tasks = 20
    samples_per_task = total_samples // num_tasks

    # Results collector
    results = []
    def collect_result(value):
        results.append(value)
        print(f"Task complete. Total collected: {len(results)}/{num_tasks}")

    with mp.Pool(processes=mp.cpu_count()) as pool:
        for _ in range(num_tasks):
            # Non-blocking: fires off 20 parallel tasks
            pool.apply_async(monte_carlo_task, (samples_per_task,), callback=collect_result)

        print("Master process: All tasks dispatched. Waiting for final results...")
        pool.close() # No more tasks
        pool.join()  # Wait for all workers to exit

    # Final reduction (HPC logic)
    pi_est = (4.0 * sum(results)) / total_samples
    print(f"Final Pi Estimate: {pi_est}")

Master process: All tasks dispatched. Waiting for final results...
Task complete. Total collected: 1/20
Task complete. Total collected: 2/20
Task complete. Total collected: 3/20
Task complete. Total collected: 4/20
Task complete. Total collected: 5/20
Task complete. Total collected: 6/20
Task complete. Total collected: 7/20
Task complete. Total collected: 8/20
Task complete. Total collected: 9/20
Task complete. Total collected: 10/20
Task complete. Total collected: 11/20
Task complete. Total collected: 12/20
Task complete. Total collected: 13/20
Task complete. Total collected: 14/20
Task complete. Total collected: 15/20
Task complete. Total collected: 16/20
Task complete. Total collected: 17/20
Task complete. Total collected: 18/20
Task complete. Total collected: 19/20
Task complete. Total collected: 20/20
Final Pi Estimate: 3.14616
